<a href="https://colab.research.google.com/github/Saadman53/Thesis-Human-Activity-Recognition/blob/main/(UTD)Feature_Extraction_using_Alexnet(Prewitt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')
gdrive_path = "drive/My Drive/Dataset/UTD MHAD/utd_mhad_mei_861x277x277.mat"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

import torchvision
import torchvision.transforms as transforms



import random
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report


from torch.utils.data import DataLoader, ConcatDataset
# from torchvision import transforms
from sklearn.model_selection  import StratifiedKFold

import torch.optim.lr_scheduler as lr_scheduler
%matplotlib inline

import scipy.io as sio
torch.autograd.set_detect_anomaly(True)


from functools import partial
from typing import Any, Optional

import torch
import torch.nn as nn


In [19]:
dat = sio.loadmat(gdrive_path)

In [20]:
dat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mei'])

In [21]:
X = dat['mei'][0][0]
y = dat['mei'][0][2].ravel()
filename = dat['mei'][0][1].ravel()

In [22]:
def alexnet_feature_extraction(X):
  'Convert to 3 channel'
  data = torch.empty((X.shape[0],3,227,227),dtype=torch.float32)
  for i in range(X.shape[0]):
    t = torch.tensor(X[i], dtype = torch.float32).reshape((1,227,227))
    t = t.expand(3,*t.shape[1:])
    data[i] = t
  data = torch.tensor(data)
  'Apply necessary transformations'
  data_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
  ])
  X_transformed = data_transforms(data)

  'Instantiate the model'
  alexnet = torchvision.models.alexnet(pretrained=True)
  alexnet.classifier = torch.nn.Sequential(*list(alexnet.classifier.children())[:-4]) #keeping only the first layer

  'Create the new dataset'
  dataset = torch.empty((X.shape[0],1,4096),dtype=torch.float32)
  for i in range(data.shape[0]):
    batch_data = torch.unsqueeze(X_transformed[i],0)
    dataset[i] =  alexnet(batch_data)
    print(i*100/data.shape[0])
  return dataset


In [23]:
X = alexnet_feature_extraction(X)

0.0
0.11614401858304298
0.23228803716608595
0.34843205574912894
0.4645760743321719
0.5807200929152149
0.6968641114982579
0.8130081300813008
0.9291521486643438
1.0452961672473868
1.1614401858304297
1.2775842044134726
1.3937282229965158
1.5098722415795587
1.6260162601626016
1.7421602787456445
1.8583042973286876
1.9744483159117305
2.0905923344947737
2.2067363530778166
2.3228803716608595
2.4390243902439024
2.5551684088269453
2.671312427409988
2.7874564459930316
2.9036004645760745
3.0197444831591174
3.1358885017421603
3.252032520325203
3.368176538908246
3.484320557491289
3.6004645760743323
3.7166085946573753
3.832752613240418
3.948896631823461
4.065040650406504
4.181184668989547
4.29732868757259
4.413472706155633
4.529616724738676
4.645760743321719
4.761904761904762
4.878048780487805
4.994192799070848
5.110336817653891
5.2264808362369335
5.342624854819976
5.458768873403019
5.574912891986063
5.691056910569106
5.807200929152149
5.923344947735192
6.039488966318235
6.155632984901278
6.271777003

In [24]:
print(X.shape)

torch.Size([861, 1, 4096])


In [25]:
dic = {'data': X.detach().numpy(), 'labels': y, 'filename': filename}
sio.savemat("drive/My Drive/Dataset/UTD MHAD/utd_alexnet_extracted_4096.mat", dic)